# Python Learn by Doing: Climate Change Indicators and Trend Analysis

**Developed By:** Dr. Kerrie Geil, Mississippi State University

**Original Development Date:** May 2024

**Package Requirements:** xarray, netcdf4, numpy, pandas, scipy, matplotlib, jupyter (and cartopy for one of the Your Turn assignments)

**Hardware requirements:** 100MB available storage on your machine, 8GB RAM

**Links:** **[OSF project link](https://osf.io/zhpd5/)**, [link to this notebook](https://github.com/kerriegeil/MSU_py_training/blob/main/learn_by_doing/climate_change_indicators/climate_change_indicators_AgERA5.ipynb)

**Description:**

This notebook helps the learner build intermediate python programming skills through data query, manipulation, analysis, and visualization. Learning will be centered around computing ETCCDI climate change indices and determining the statistical significance of change. The notebook is aimed at learners who already have intermediate programming skills and some knowledge of statistics. 

-----

# Collection of Useful Links

- all python trainings modules developed by Dr Kerrie Geil can be found in the [OSF MSU_py_training project](https://osf.io/zhpd5/) on the Open Science Framework platform. There you will find links to the most updated versions of the training notebooks on github as well as any required data files.
- [kerrie's github repo](https://github.com/kerriegeil/MSU_py_training) contains python codes but not data files. This repo is linked to the OSF MSU_py_training project above
- [documentation page for every version of Python](https://www.python.org/doc/versions/)
- every version of Python also includes a tutorial e.g. [The Python Tutorial v3.12.3](https://docs.python.org/release/3.12.3/tutorial/index.html)
- this is not technically a link, but Google is your friend! If there is something you want to learn how to do with python, just Google it and click through the search results to see if you can find an answer. A Google search will often return results from online forums like Stack Overflow where someone has asked the same question and received several answers. This same principle works for figuring out conda and jupyter things. Chances are the answer is a quick web search away
- [jupyter markdown cheat sheet](https://notebook.community/tschinz/iPython_Workspace/00_Admin/CheatSheet/Markdown%20CheatSheet)
- [documentation on climate change indicators from the Expert Team on Climate Change Detection and Indices (ETCCDI)](https://etccdi.pacificclimate.org/index.shtml)
- [climdex.org maps and data files](https://www.climdex.org/learn/indices/) for many climate change indices and their trends
- [account registration page](https://cds.climate.copernicus.eu/user/register?destination=%2Fcdsapp%23!%2Fhome) for the [climate data store](https://cds.climate.copernicus.eu/cdsapp#!/home) where you can find tons of free research quality climate data
- [xarray documentation](https://docs.xarray.dev/en/stable/) which includes the api reference, getting started guide, user guide, and developer info

# Introduction to Climate Change Indicators

Climate change indicators (also known as climate change indices) are quantitative measures of some aspect of the climate that can be tracked over time and used to robustly assess climate changes. Good climate change indicators use high quality observational data that is consistent over time (or has been adjusted/homogenized in time<sup>1</sup>) and incorporate statistical methods, if necessary, that reduce the impact of possible data inconsistencies. 

In this notebook we'll look at a few climate change indicators developed by the [Expert Team on Climate Change Detection and Indices (ETCCDI)](https://etccdi.pacificclimate.org/index.shtml) which is a sub-project of [Climate and Ocean Variability, Predictability and Change (CLIVAR)](https://www.clivar.org/), one of the core projects of the [World Climate Research Program's (WCRP)](https://www.wcrp-climate.org/).

The ETCCDI developed a set of [27 core indices](https://etccdi.pacificclimate.org/list_27_indices.shtml) using temperature and precipitation observational data. We'll calculate 5 of these:
- Monthly Maximum Value of Daily Minimum Temperature (TNx)
- Annual Total Precip Amount Over 99th Percentile of Wet Days (R99pTOT)
- Monthly Maximum Consecutive 5-day Precipitation (Rx5day)
- Maximum Length of Consecutive Dry Days (CDD)
- Growing Season Length (GSL)

**Disclaimer:** This notebook is intended for python programming learning only. The data quality checking and calculation of ETCCDI climate change indices in this notebook may differ from the ETCCDI published instructions for simplicity and/or relevance to our learning goals. Learners wanting to compute the indices according to the exact ETCCDI instructions should consult the [ETCCDI documentation](https://etccdi.pacificclimate.org/index.shtml). The documentation suggests using the [RClimDex software package](https://github.com/ECCC-CDAS/RClimDex.git) written in R to calculate ETCCDI climate change indices. Another option would be to use pre-calculated indices based on multiple gridded datasets available at **[climdex.org](https://www.climdex.org/)**, where you can also find a similar software package for calculating the indices on a dataset of your choice.   


Footnotes:
1. Time-dimension inhomogeneities in meteorological observations are due to things like station relocation and changes in instrumentation. For more info, ETCCDI has compiled examples of these inhomogeneities in [Classic Examples of Inhomogeneities in Climate Datasets](https://etccdi.pacificclimate.org/docs/Classic_Examples.pdf)



# Importing Python Packages and Defining Your Workspace


In [ ]:
# importing all the python packages we will need here

import xarray as xr
import numpy as np
import pandas as pd
import scipy.stats as ss
import numpy.testing as npt
import os
from urllib.request import urlretrieve
import matplotlib.pyplot as plt

In [ ]:
# create a folder for data downloads
if not os.path.exists('./data'):
    os.makedirs('./data')

In [ ]:
# filenames to save data to
pr_f='data/prcp_AgERA5_Starkville_Daily_1979-2023.nc'
tmax_f='data/tmax_AgERA5_Starkville_Daily_1979-2023.nc'
tmin_f='data/tmin_AgERA5_Starkville_Daily_1979-2023.nc'

# Obtaining the climate data

For the climate change indices covered in this notebook we will need the following data over many data years:

variable abbrev. | description | frequency | units 
---|---|---|---
tmin | minimum surface air temperature | daily | C 
tmax | maximum surface air temperature | daily | C 
prcp | accummulated precipitation | daily | mm/day 

We'll use the AgERA5 dataset developed by the [European Centre for Medium-range Weather Forecasts (ECMWF)](https://www.ecmwf.int/) available from the [Copernicus Climate Data Store (CDS)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-agrometeorological-indicators?tab=overview). The data variables in the table above were downloaded from the AgERA5 dataset by the instructor in May 2024, pre-processed, and placed in the [MSU_py_training project on the Open Science Framework (OSF) platform](https://osf.io/zhpd5/). The reason we use OSF to store these files is to guarantee permanent availability of the exact same data files for this training module into the future.

Data file pre-processing includes the following and was completed for you to save time:
1) Using the Climate Data Store (CDS) API to download AgERA5 precipitation and 2m min and max temperature. You need a free CDS account and to install the cdsapi python package as well as python dask to do this. The data downloads as one .tar.gz file per year per variable.
2) Unpacking the .tar.gz files. Each g-zipped archive unpacks to one netcdf file per day per variable (many files).
3) Converting temperature units from K to C
4) Converting longitude coordinates from 0 to 360 to -180 to 180
5) Consolidating all the daily files into one single netcdf file per variable that contains all times.

We won't cover this process in further detail here but if you are interested, the scripts used to pre-process the data are available on Kerrie's github. Step 1-2 is performed in [get_AgERA5_daily_parallel.py](https://github.com/kerriegeil/MSU_py_training/blob/main/learn_by_doing/climate_change_indicators/get_datasets/get_AgERA5_daily_parallel.py) and steps 3-5 are performed in [concat_AgERA5.sh](https://github.com/kerriegeil/MSU_py_training/blob/main/learn_by_doing/climate_change_indicators/get_datasets/concat_AgERA5.sh).

<br>
<font color="green"><b>
**NOTE: You only need to run the following urlretrieve cells once. The data files will then be located on your computer. These files total approximately 4MB in size.**
</b></font> 

In [ ]:
# data near Starkville, MS

# precip
url='https://osf.io/aes24/download' # url to the data
urlretrieve(url, pr_f) # download and save data

In [ ]:
# tmax
url='https://osf.io/d3hgm/download' # url to the data
urlretrieve(url, tmax_f) # download and save data

In [ ]:
# tmin
url='https://osf.io/pjzsx/download' # url to the data
urlretrieve(url, tmin_f) # download and save data

# Introduction to python xarray 

"Xarray introduces labels in the form of dimensions, coordinates and attributes on top of raw Numpy-like multidimensional arrays, which allows for a more intuitive, more concise, and less error-prone developer experience"

   >-**[xarray documentation](https://docs.xarray.dev/en/stable)**

This package is most powerful when working with multidimensional data in netcdf format and especially with netcdfs that are written using Climate and Forecast Metadata Conventions ([CF Metadata Conventions](https://cfconventions.org)). Xarray can also handle zarr, tiff, csv, hdf, and grib files but may require additional dependencies to be installed. 

Xarray builds upon Numpy, Pandas, Scipy, netCDF4, Matplotlib and more. It also integrates well with Dask for parallel computing. Xarray is under active community development and pushes new updates approximately monthly.

The core data structures used by xarray are called the *DataArray* and the *Dataset*. DataArrays are N-dimensional arrays of a single data variable with dimension, coordinate, and attribute labels. Datasets contain one or more DataArrays that share one or more dimensions and coordinates. Each variable in a Dataset has its own attributes and the Dataset itself can have its own attributes as well (which come from the file attributes in each netcdf). 

Details of all xarray functions (including what parameters to include as function inputs and what each function returns) can be found in the **[xarray API reference](https://docs.xarray.dev/en/stable/api.html)**. Xarray has pretty great documentation with usage examples, definitely check the **[xarray getting started](https://docs.xarray.dev/en/stable/getting-started-guide/index.html)** and **[xarray user guide](https://docs.xarray.dev/en/stable/user-guide/index.html)** documentation for help as you are learning. If you are stuck on something, stack overflow and xarray's issue documentation on github is also useful. I personally often end up at those sites from google searches "python xarray how to ___". 

Let's read in precipitation data from a netcdf file and look at some xarray capabilities.


    

In [ ]:
# the netcdf files we're using in this notebook have 1 data variable per file
# read data into a DataArray structure

pr = xr.open_dataarray(pr_f)
pr

With xarray, when we print a variable, instead of getting the values of that variable what we get (usually) is a view of all the metadata labels that are attached to the variable. The information above shows us that our pr data is the daily total precipitation aggregated from midnight to midnight local time each day, has units of mm per day, and is called 'prcp' in the netcdf file.  

We can also see that the data has 3 dimensions (time, lat, lon), the length of each dimension, and that each dimension is a "coordinate", which are essentially additional labels. Click on the paper and data stack icons to the right of each coordinate. Using the paper icon, you can see that each coordinate has its own attributes (standard_name, units, etc.). Using the data stack icon, you can see that each coordinate is also an array of values, similar to an index in Pandas. The beauty of coordinates is that they allow us to easily select a subset of the data variable using labels that correspond to the coordinate values. 

Definitions for xarray terminology such as DataArray, Dataset, variable, dimension, coordinate, attribute can all be found in xarray's user guide on the **[xarray terminology page](https://docs.xarray.dev/en/stable/user-guide/terminology.html)**.

Let's start looking at how to access labels and select/manipulate our data using labels. 

In [ ]:
# access the coordinate information (labels/values, attributes) attached to a variable
# same can be done with lon and time
# this is the same info we can see above by clicking on the coordinate icons
pr.lat

In [ ]:
# attributes are always stored as dictionaries which contain items in key:value pairs
# access all the attributes attached to a variable or a coordinate with .attrs

# see dictionary of attributes attached to the variable pr
print(pr.attrs)

# see dictionary of attributes attached to the coordinate lat
pr.lat.attrs

Dictionaries are not xarray-specific, they are a python data structure. Here are some good links to more info about dictionaries and other python data structures: **[realpython.com/python-dicts/](https://realpython.com/python-dicts/)** and **[docs.python.org/3/tutorial/datastructures](https://docs.python.org/3/tutorial/datastructures.html#)**

In [ ]:
# since attributes are stored in dictionaries we can use dictionary syntax to access the items 

# list all the pr attibutes names (keys) present in the dictionary
print(pr.attrs.keys())

# access the value of the pr 'units' attribute 
print(pr.attrs['units'])

# access the value of the latitude 'units' attribute
pr.lat.attrs['units']

Now let's start selecting data from the pr array based on its time coordinate labels.

This particular data consists of only 1 data point at lat=33.5, lon=-88.8 (Starkville, MS). We don't need these singleton dimensions so let's remove them. Notice below the singleton dimensions are removed from the variable, but the lat/lon coordinate metadata sticks around in case we need it later (this would be useful, for example, if we wanted to plot the lat/lon location on a map).

In [ ]:
# remove singleton dimensions
pr=pr.squeeze()
pr

In [ ]:
# select data for 1 May 1982
pr.sel(time='1982-05-01')

In [ ]:
# select data for all of 2020
pr.sel(time=slice('2020-01-01','2020-12-31'))

what if we want to see the data value but not the extra metadata labels on a particular day?


In [ ]:
# .data on an xarray object will return the underlying numpy array of data without labels
pr.sel(time='1982-05-01').data

In [ ]:
# add data from multiple days together
# sum all precip in 2020
pr.sel(time=slice('2020-01-01','2020-12-31')).sum()

What if we want to select data from the array based on some sort of criteria? .where is a great solution

Important to note that xarray.where and xarray.DataArray.where are different. Here we're using **[xarray.DataArray.where](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.where.html)**. We'll cover **[xarray.where](https://docs.xarray.dev/en/stable/generated/xarray.where.html)** later

In [ ]:
# selecting with .where
# find pr > 3 inches/day (3 inches/day * 2.54 cm/inch * 10 mm/cm = 76.2 mm/day)

# select pr where greater than 76.2 and drop all other days from the array
pr.where(pr>76.2,drop=True)

In [ ]:
# if you don't drop=True xarray keeps the data dimensions 
# and automatically fills the days that don't meet the criteria with nan
pr.where(pr>76.2)

In [ ]:
# do the same thing but return the dates that meet the criteria, not the pr values
times_heavy=pr.where(pr>76.2,drop=True).time
times_heavy

In [ ]:
# you can also select from a DataArray using a list or array of labels
# select from pr with an array of time labels
pr.sel(time=times_heavy)

If the time coordinate is datetime objects like we have here, we can do some real fancy stuff. Xarray implements a lot of the useful features from pandas with the **[xarray .dt datetime accessor](https://docs.xarray.dev/en/latest/generated/xarray.core.accessor_dt.DatetimeAccessor.html)**.

In [ ]:
pr.time

In [ ]:
pr.time.dt.day

In [ ]:
pr.time.dt.isocalendar()

**[xarray .dt datetime accessor](https://docs.xarray.dev/en/latest/generated/xarray.core.accessor_dt.DatetimeAccessor.html)** is very helpful when used in combination with **[xarray.DataArray.resample](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.resample.html)** or **[xarray.DataArray.groupby](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.groupby.html)** to easily resample or group your data, which we'll get to here shortly 

You can find all the different methods (e.g. .sum(), .mean(), etc) that are possible to use on xarray resample and groupby objects here: **[xarray.core.resample.DataArrayResample](https://docs.xarray.dev/en/latest/generated/xarray.core.resample.DataArrayResample.html#xarray.core.resample.DataArrayResample)** and **[xarray.core.groupby.DataArrayGroupBy](https://docs.xarray.dev/en/latest/generated/xarray.core.groupby.DataArrayGroupBy.html)**

In [ ]:
# resample to annual sum 
# and make the time coordinate label the last day of the year 
# (YE = year end)
pr_annual = pr.resample(time='YE').sum()
pr_annual

In this case some of our variable attributes may no longer make sense. Sometimes this happens when we operate on our data with certain functions (like resample). Or sometimes, functions that aggregate or reduce your data will actually delete coordinates and variable attributes to avoid leaving metadata that is no longer accurate. In these situations, we manually update or add new metadata labels in cases where they get deleted entirely.

When and why would we want to update coordinates or variable attributes like standard_name, units, etc? 

Every little bit of metadata (like attributes) that are attached to your variables don't need to be 100% accurate and up to date at all times. But, it becomes important to update this info in at least 2 situations: 
1) if you use a package for plotting that automatically places labels on figures based on the variable attributes and coordinate labels/values present (xarray plotting does this) or 
2) if you write a variable out to a new netcdf file you'll want to make sure all the metadata attached to your variable is correct (good data management)  

In [ ]:
# modifying attributes

# replace a variable attribute value
pr_annual.attrs['units']='mm/year'

# delete a whole variable attribute
del pr_annual.attrs['temporal_aggregation']

# knock the last 10 characters off a variable attribute
pr_annual.attrs['long_name']=pr_annual.attrs['long_name'][:-10]

# look at our new attributes
pr_annual

In [ ]:
# another resample example
# resample to seasonal mean
# 'QS' means make the time label for each season at the start of the season (quarter start)
# 'JAN' means make start the seasonal aggregates with January: JFM, AMJ, JAS, OND
pr.resample(time='QS-JAN').sum()

.resample and .groupby can be used to easily calculate anomalies

Let's make monthly pr anomalies. We calculate monthly mean pr and subtract the monthly long term mean (climatology) pr.

This means, for example, Jan 1979 mean pr - Jan (1979-2023) mean pr, Feb 1979 mean pr - Feb (1979-2023) mean pr, and so on for each monthly data point

In [ ]:
# first we need mean the monthly mean pr values, use .resample
pr_monthly=pr.resample(time='MS').mean()
pr_monthly

We'll use **[xarray.DataArray.groupby](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.groupby.html)** for the rest of the calculation. Before we proceed, let's take a look at .groupby. If you are familiar with Pandas, this will look very similar since xarray .groupby is based on Pandas .groupby

In [ ]:
# .groupby returns an object that can be iterated over in the form (label,grouped_data) pairs
pr_monthly.groupby(pr_monthly.time.dt.month)

if we think in terms of xarray dimensions and coordinates, this is similar to getting a new dimension named month that is also a coordinate with values/labels 1-12 

In [ ]:
# how to access the group labels (dictionary-like syntax)
pr_monthly.groupby(pr_monthly.time.dt.month).groups.keys()

In [ ]:
# how to access the group of data assigned to a label (dictionary-like syntax)
# the following shows the data associated with label (month) 5
label=5
pr_monthly.groupby(pr_monthly.time.dt.month)[label]

If you look at all the time labels (data stack icon for the time coordinate) you'll see this group is all the May data values in the pr_monthly timeseries. 45 years worth of May values. We can calculate the long term May mean pr by taking the mean of this group of data

In [ ]:
pr_monthly.groupby(pr_monthly.time.dt.month)[label].mean()

We can calculate the long term mean for each month of the year (the groups) by excluding the [label] part from above. The value of the 5th number in the returned array below should be the same as above. Notice also below the new dimension and coordinate named 'month' and its coordinate labels 1-12. 

In [ ]:
pr_monthly.groupby(pr_monthly.time.dt.month).mean()

These new labels from .groupby are what makes .groupby so "smart" when we actually compute the anomalies. We'll be taking each monthly mean pr value in the timeseries and subtracting the appropriate long term mean value, all without any loops to slow us down.

In [ ]:
# calculate anomalies, putting it all together

# the first term is 12 groups (months) each containing 45 data values (1 value per year)
# the second term is 12 groups (months) each containing 1 value (long term monthly mean) 

# based on the labels present, for each group .groupby is smart enough to broadcast 
# the 1 value in the second term to each of the 45 values in the first term
# in other words, the appropriate long term mean value is subtracted from every monthly data value in the time series
# no loops (or nested loops) needed

pr_anom= pr_monthly.groupby(pr_monthly.time.dt.month) - pr_monthly.groupby(pr_monthly.time.dt.month).mean()
pr_anom

Now, we'll make a quick figure of our pr daily timeseries and our monthly pr anomalies. 

Here's a link to **[xarray plotting basics](https://docs.xarray.dev/en/latest/user-guide/plotting.html)** 

Xarray plotting is based on matplotlib. here are some useful links: **[matplotlib documentation](https://matplotlib.org/stable/index.html)** and **[short matplotlib tutorial](https://matplotlib.org/stable/tutorials/pyplot.html)** and **[matplotlib api reference](https://matplotlib.org/stable/api/index.html)**

Remember, at the beginning of this notebook we imported matplotlib.pyplot as plt, so that's where plt is coming from below

We are using a combination of xarray plotting (**[xr.DataArray.plot](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.plot.html)**) and **[matplotlib.pyplot](https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html)** which work seamlessly together

In [ ]:
# plotting

fig=plt.figure(figsize=(15,2))
pr.plot()
plt.title('daily pr at Starkville, MS')
plt.show()

fig=plt.figure(figsize=(15,2))
pr_anom.plot()
plt.axhline(y=0,color='grey',linestyle='dashed',linewidth=0.5)
plt.title('monthly mean pr anomaly at Starkville, MS')
plt.ylabel('pr (mm/day)')
plt.show()

Thus far we've been working with data in xarray's DataArray structure. We opened our data file with **[xarray.open_dataarray](https://docs.xarray.dev/en/latest/generated/xarray.open_dataarray.html)** 

Sometimes there are multiple variables in a netcdf file as well as file attributes that we need (different from variable or coordinate attributes). In this situation you would read the data file into an xarray Dataset (instead of a Data Array) using **[xarray.open_dataset](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html)**. 

This doesn't really apply to us here since our data files only have 1 variable each and we don't need the file attributes, but we'll go through an example of how to read data into a Dataset below anyway.

In [ ]:
# .open_dataset instead of .open_dataarray
ds = xr.open_dataset(pr_f)
ds

Notice you can now see the file attributes called 'history' and 'NCO' (sometimes also called global attributes). If this file had more than one variable they would appear in the Data variables section under prcp.

Similar to how you can access coordinate information from a data variable with varname.coordname, you can access the data variables in a dataset with dataset.varname like below 

In [ ]:
pr=ds.prcp
pr

# Data Cleaning

The minimum quality control procedures suggested by ETCCDI are as follows.

Replace data value with Nan for:
- user-defined missing values (i.e -9999-->Nan)
- daily precip values less than 0
- daily max temperature less than daily minimum temperature
- daily temperature greater than 70C (158F) or less than -70C (-94F)
- leap days (i.e Feb 29th)
- impossible dates (i.e. 32nd March, 12th June 2042)
- non-numeric values
- daily temperature outliers (i.e. 3-5 times the standard deviation from the mean value for each calendar day)


Addressing each of these items below...

In [ ]:
# first let's start with a fresh read of all the data we'll be using
pr = xr.open_dataarray(pr_f)
tx = xr.open_dataarray(tmax_f)
tn = xr.open_dataarray(tmin_f)

pr=pr.squeeze()
tx=tx.squeeze()
tn=tn.squeeze()
pr

### nan for user-defined missing values (i.e -9999-->Nan)

xr.open_dataset does this for you. 


Notice in the variable attributes above there is no _FillValue=-9999., which the value stored for missing data in the netcdf. This is because xarray automatically replaces the _FillValue with nan.

### how much data is missing to begin with?

before we move on to the rest of the data cleaning, it's usually a good idea to get acquainted with how many missing data values there are

We'll do a quick look here and then look deeper later


In [ ]:
print('total of',pr.isnull().sum().data,'missing pr values')
print('total of',tx.isnull().sum().data,'missing tx values')
print('total of',tn.isnull().sum().data,'missing tn values')

Good news! We don't really have to worry about missing data since there is only 1 day missing in one of our data variables

### nan for daily precip values less than 0

In [ ]:
# are there any negatives?
(pr<0).data.sum()

### nan for daily max temperature less than daily minimum temperature

In [ ]:
# is tx ever less than tn?
(tx<tn).data.sum()



In [ ]:
# # if there were any tx<tn this is how we would fix it
# # where tx<tn fill both tx and tn with nan
# tx=xr.where(tx<tn,np.nan,tx)
# tn=xr.where(tx<tn,np.nan,tn)

# # is tx ever less than tn now?
# (tn>tx).data.sum()

### nan for daily temperature greater than 70C (158F) or less than -70C (-94F)

In [ ]:
# is tx>70C, tx<-70C, tn>70C, or tn<-70C?
((tx>70)|(tx<-70)).data.sum(), ((tn>70)|(tn<-70)).data.sum()

### leap days (i.e Feb 29th)

To make our lives easier later on instead of filling leap days with nan, we'll drop leap days from the timeseries entirely. 

First let's double check that the time dimension is the same for all of our data arrays 

More information on using python assert and numpy.testing for code debugging: **[realpython.com/python-assert-statement](https://realpython.com/python-assert-statement/)** and **[numpy.testing API reference](https://numpy.org/doc/stable/reference/routines.testing.html)**


In [ ]:
# check time dims are all the same
assert list(pr.time.data)==list(tx.time.data), 'pr.time and tx.time are not equal'
assert list(pr.time.data)==list(tn.time.data), 'pr.time and tn.time are not equal'

# another way to do the same thing without having to convert data structure is with numpy.testing
npt.assert_array_equal(pr.time,tx.time,'npt error: pr.time and tx.time are not equal')
npt.assert_array_equal(pr.time,tn.time,'npt error: pr.time and tn.time are not equal')

Notice we're now using **[xarray.where](https://docs.xarray.dev/en/stable/generated/xarray.where.html)** (a core function of the xarray package) below which accepts different parameters than **[xarray.DataArray.where](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.where.html)** (a method that operates on an xarray data array object) that we used earlier 

In [ ]:
# create a boolean array of dim 'time' where leap days are True and all other days are False
isleapday=xr.where((pr.time.dt.day==29) & (pr.time.dt.month==2),True,False)
isleapday

In [ ]:
# how many leap days are there?
isleapday.data.sum()

In [ ]:
# drop leap days from each data variable
pr=pr.where(~isleapday,drop=True)  # ~ means not (where isleapday is not True)
tx=tx.where(isleapday!=True,drop=True)  # !=True is another way to do the same thing 
tn=tn.where(isleapday==False,drop=True)  # ==False is a third way of doing the same thing

pr  # should be 11 days shorter

### nan for impossible dates (i.e. 32nd March, 12th June 2042)

This data has datetimes for the time dimension. If there were impossible dates, xarray would have had a problem at the open_dataarray statement. So we know there are no impossible dates present.

Missing dates, on the other hand, would not throw an error. But we can check if there are missing dates just by looking at the length of the time dimension. We have 45 years of daily data, now without leap days. 

45years * 365days = 16425days

In [ ]:
len(pr), len(tx), len(tn)

### nan for non-numeric values

Similarly, use of netcdf and xarray ensures that there are no non-numeric values or mixed numeric data types per variable. Each variable in the data file is of one data type (e.g. float) and if there were a non-float value present there would have been an error already. We can be assured the data we've read is all float. For more on data types: **[data types tutorial](https://realpython.com/python-data-types/)** and **[min/max values of different data types](https://jakevdp.github.io/PythonDataScienceHandbook/02.01-understanding-data-types.html#NumPy-Standard-Data-Types)**

In [ ]:
pr.dtype, tx.dtype, tn.dtype

### daily temperature outliers (i.e. 3-5 times the standard deviation from the mean value for each calendar day)

We'll use 5x standard deviation to identify outliers

Using .dt.dayofyear here might seem tempting to calculate the standard dev for each day for example tx.groupby(tx.time.dt.dayofyear).mean('time'), but it's going to give you the wrong results! Let's take a look at why

Use .groupby with .dt.dayofyear and look at the data for day of year group 60 and day of year group 61

.dt.dayofyear determines the day of year specific to each year, so in leap years day 60 will be 29 Feb and in non-leap years 1 Mar

Similarly, in leap years day 61 will be 1 Mar but in non-leap years 2 Mar, and so on for every day of the year >= 60

In some cases, maybe this is desirable, but in our case we want the day of year to be 1-365 regardless of leap year or non-leap year

In [ ]:
# what data is in the dayofyear 60 group?
tx.groupby(tx.time.dt.dayofyear)[60] # only 34 march 1st?

In [ ]:
# what data is in the dayofyear 61 group?
tx.groupby(tx.time.dt.dayofyear)[61] # mix of mar 1 and mar 2?

In [ ]:
tx.groupby(tx.time.dt.dayofyear).count()

In [ ]:
# code we could have used to double checked ourselves

# is there data from only 1 date (e.g. all 1 Mar) in each group?
ndates=[]
for label,group in tx.groupby(tx.time.dt.dayofyear):
    ndates.append(len(np.unique(group.time.dt.day)))
print('groups contain',np.unique(ndates),'dates, expected 1')

# do all groups have the same number of days?
print('groups have',np.unique(tx.groupby(tx.time.dt.dayofyear).count()),'numbers, expected 45')

Sometimes things that are supposed to make our lives easier, just don't work out. 

We need to write a new coordinate with values 1-365 (we'll call it doy) and then use .groupby with this new index so we can ignore leap days entirely, we should get 45 march 1st, 45 march 2nd, etc for every date (and no leap days because we've dropped them)

In [ ]:
# create a list 16425 numbers long where day of the year without leap days (1-365) is repeated 45 times
doy=list(range(1,366))*45
doy[363:367],len(doy)

In [ ]:
tx.coords['doy']=('time',doy)
tn.coords['doy']=('time',doy)
tx

In [ ]:
# is there data from only 1 date (e.g. all 1 Mar) in each group?
ndates=[]
for label,group in tx.groupby(tx.doy):
    ndates.append(len(np.unique(group.doy)))
print('tx groups contain',np.unique(ndates),'dates, expected 1')

ndates=[]
for label,group in tn.groupby(tn.doy):
    ndates.append(len(np.unique(group.doy)))
print('tn groups contain',np.unique(ndates),'dates, expected 1')

# do all groups have the same number of days?
print('tx groups have',np.unique(tx.groupby(tx.doy).count()),'numbers, expected 45')
print('tn groups have',np.unique(tn.groupby(tn.doy).count()),'numbers, expected 45')

In [ ]:
# find the long term mean for each day of the year
tx_daily_mean=tx.groupby(tx.doy).mean()
tn_daily_mean=tn.groupby(tn.doy).mean()
tx_daily_mean

In [ ]:
# find the standard deviation for each day of the year
tx_stddev=tx.groupby(tx.doy).std()
tn_stddev=tn.groupby(tn.doy).std()
tx_stddev

In [ ]:
# define daily outlier temperature as exceeding the mean +/- 5 times standard deviation
tx_outlier_upper, tx_outlier_lower=(tx_daily_mean+tx_stddev*5), (tx_daily_mean-tx_stddev*5)
tn_outlier_upper, tn_outlier_lower=(tn_daily_mean+tn_stddev*5), (tn_daily_mean-tn_stddev*5)
tx_outlier_upper

In [ ]:
# are there any outliers?
print('tx',(tx.groupby(tx.doy)>tx_outlier_upper).data.sum(), (tx.groupby(tx.doy)<tx_outlier_lower).data.sum())
print('tn',(tn.groupby(tn.doy)>tn_outlier_upper).data.sum(), (tn.groupby(tn.doy)<tn_outlier_lower).data.sum())

Out of all the data cleaning steps above, the only step that resulted in a change to the original data arrays was dropping leap days. This likely would not hold true for AgERA5 data at other locations (e.g. not Starkville) or, of course, for data from other sources.

### another look at nans

We already know that tx is the only data variable with a missing data point. But if we had more missing data, it would be helpful to know how many nans per month there are since many of our climate change indicators group data by month. We'll write a custom function to do this.

more on writing functions here: **[realpython.com/defining-your-own-python-function](https://realpython.com/defining-your-own-python-function/)**

more on pandas MultiIndexing here: **[pandas MultiIndex and advanced indexing](https://pandas.pydata.org/docs/user_guide/advanced.html)**

In [ ]:
# a function that sums the number of nans in each month of data
def get_nans_per_month(data_in):
    month_groups=pd.MultiIndex.from_arrays([data_in.time['time.year'].data,data_in.time['time.month'].data])
    data_in.coords['month_groups']=('time',month_groups)    
    nancount=data_in.isnull().groupby('month_groups').sum()
    return nancount

In [ ]:
# call our function
nan_per_month=get_nans_per_month(tx.copy())
nan_per_month

we can see the one missing data point we were expecting

Let's plot nan_per_month. While we're at it, we'll make a proper datetime array for the x-axis values with **[pandas.date_range](https://pandas.pydata.org/docs/reference/api/pandas.date_range.html)**

In [ ]:
# plot it

# create datetimes for the x axis
# MS means month start for the monthly time labels e.g. 1979-01-01, 1979-02-01
# if we used ME instead (month end) we'd get time labels like 1979-01-31, 1979-02-28
# either is ok, MS is a personal preference
time_months=pd.date_range(tx.time.data[0],tx.time.data[-1],freq='MS')

# plot
fig=plt.figure(figsize=(10,2))
plt.plot(time_months,nan_per_month)
plt.title('tmax, number of nans per month')
plt.tight_layout()
plt.show()

and which specific day is it when we are missing data?

In [ ]:
tx.where(tx.isnull(),drop=True).time

# Calculate climate change indicators

### Monthly Maximum Value of Daily Minimum Temperature (TNx)

- max(each month of daily minimum temperature values)

Here we are inputting daily data and pulling out 1 value per month.

In [ ]:
# create coordinate labels that we can use to group data in every month of the timeseries

# this is similar to how we found nans per month
# create an index for every month in the timeseries so we can use .groupby with it later
month_groups=pd.MultiIndex.from_arrays([tn.time['time.year'].data,tn.time['time.month'].data])

# add the month_groups index as a new coordinate 
# the month_groups coordinate will allow us to groupby every month 
# each day of data in any particular month will be associated with a label in the month_groups coordinate
tn.coords['month_groups']=('time',month_groups)    
tn

In [ ]:
# now groupby month and find the maximum value of each month
# this should return 1 value for every month, 45 years x 12 months = length 540
TNx=tn.groupby('month_groups').max()
TNx

Notice we lose the original 'time' coordinate containing datetimes. This is because we have reduced our data array with .groupby().max(), and the original daily datetimes no longer apply, so xarray deletes the old time coordinate that no longer makes sense.

If we want, we can write a new time coordinate with monthly datetimes where the label would be the first day of each month (e.g. 1979-01-01). If you remember, we've already made these exact monthly datetimes earlier 'time_months'. We'll use time_months to write a new time coordinate to our TNx data array. We don't have to do this, but it will make plotting a figure easier.

In [ ]:
# change the month_groups dim name and coordinate labels to datetimes
TNx=TNx.rename({'month_groups':'time'})  # renaming a coordinate and dimension
TNx=TNx.drop_vars(['time_level_0','time_level_1']) # deleting leftovers from MultiIndex that we don't need
TNx.coords['time']=('time',time_months) # assign datetimes as the coordinate values/labels 
TNx

In [ ]:
# plot
fig=plt.figure(figsize=(15,2))
TNx.plot()
plt.title('monthly maximum value of daily minimum temperature (TNx)')
plt.ylabel('degrees C')
plt.show()

### Annual Total Precip Amount Over 99th Percentile on Wet Days (R99pTOT)

- annually, the sum of precipitation when precipitation is > 99th percentile of wet day precipitation in the base period 1981-2010
- where a wet day is precipitation >= 1mm

Here we first use daily data during the base period to determine the 99th percentile of wet day precipitation. Then for each year of daily data we determine if each day exceeds the threshold and calculate an annual sum of precip on days that exceed the threshold. 

In [ ]:
# slice pr to only base years
pr_baseyrs=pr.sel(time=slice('1981','2010'))
pr_baseyrs

In [ ]:
# find 99th percentile of wet day precipitation
pr_99w=xr.where(pr_baseyrs>=1.0,pr_baseyrs,np.nan).quantile(0.99)
pr_99w

In [ ]:
# fill with nan where pr doesn't meet the criteria
pr_copy=pr.copy()
pr_copy=xr.where(pr_copy>pr_99w,pr_copy,np.nan)

# then sum over each year
R99pTOT=pr_copy.groupby(pr_copy.time.dt.year).sum()
R99pTOT

In [ ]:
# plot
fig=plt.figure(figsize=(15,2))
R99pTOT.plot()
plt.title('annual total precip amount over 99th percentile on wet days (R99pTOT)')
plt.ylabel('mm')
plt.show()

### Monthly Maximum Consecutive 5-day Precipitation (Rx5day)

- max(5-day rolling mean precipitation within each month)

Here we are inputting daily data, for each month calculating the mean precipitation amount for each 5-day window of data values, then choosing the maximum of 5-day window value for each month.

In [ ]:
# we want to group by each month so we'll use our month_groups again as a new coordinate of pr
pr.coords['month_groups']=('time',month_groups) 
pr

In [ ]:
# find the maximum value of the 5-day rolling mean in each month
# let's test with one month first
window_len = 5  # days
label=(1979,5)

# use .rolling to divide the month of data up into 5-day windows
# then take the mean of each 5-day window if all 5 days in the window have finite data values (not nan)
# then find the maximum value of the above
# the result should be a single value
pr.groupby(pr.month_groups)[label].rolling(time=window_len,min_periods=window_len).mean().max()


In [ ]:
# now do the calculation for all months using a for loop
results=[]  # empty list to hold the results

# loop through each label,group pair in the groupby object
for label,data_group in pr.groupby(pr.month_groups):
    # append the result for each month to our results list
    results.append(data_group.rolling(time=window_len,min_periods=window_len).mean().max())

# the result should be 1 value for every month (45 years x 12 months = length 540)
print(len(results))

# look at the fifth item in the list (1979-05)
results[4]

In [ ]:
# even though each monthly calculation returns a single value
# that value is returned as an xarray DataArray with some metadata attached
# we have a list of xarray DataArray objects
# to get our results back into a single object we can concatenate the list of xr.DataArrays

Rx5day=xr.concat(results,dim='time') # concat on a new dimension called time
Rx5day

In [ ]:
# make the new time dimension a coordinate (supply time labels)
# we'll use time_months again to assign the time coordinate labels
Rx5day.coords['time']=('time',time_months)    

# while we're at it, assign some variable attributes
# if we use xarray plotting, some of this will show up automagically on the plot
var_atts={'standard_name':'precipitation','units':'mm/day','description':'monthly maximum of monthly 5-day rolling mean prcp'}
Rx5day.attrs=var_atts

Rx5day

In [ ]:
# plot
fig=plt.figure(figsize=(15,2))
Rx5day.plot() 
plt.title('monthly maximum consecutive 5-day precipitation (Rx5day)')
plt.show()

### Maximum Length of Consecutive Dry Days (CDD)

- annually, the maximum length of consecutive days where precipitation is < 1mm
- not looking at dry spells that span over multiple years, just cutting off the search at the end of each year

Here we are inputting daily data, determining whether each day falls under the precipitation threshold, and finding the longest period of consecutive days each year that meets the threshold requirement. 

In [ ]:
# we'll use pr again 
pr

In [ ]:
# make a mask where 1=dry and 0=wet
threshold = 1  # mm/day
dry_mask = xr.where(pr<threshold,1,0)
dry_mask

In [ ]:
# group the data by year
dry_mask_grouped=dry_mask.groupby(dry_mask.time.dt.year)

# remind ourselves what this groupby object looks like
dry_mask_grouped

In [ ]:
# use nested for loops to find the longest stretch of dry days in each year

CDD=[] # empty list to store results

# loop through each year
for year,data_group in dry_mask_grouped:
    
    counter=0
    longest_counter=0

    # loop through each day
    for iday in range(len(data_group)):

        # if it's a dry day, increment the counter
        if data_group[iday]==1: counter+=1

        # keep track of the longest consecutive amount of dry days
        if counter>longest_counter: longest_counter=counter

        # if it's not a dry day (0 or nan), start the counter over at 0
        if data_group[iday]!=1: counter=0

    # add this year's result to the list
    CDD.append(longest_counter)

    # reset counters
    longest_counter=0
    counter=0

# look at the results for first 5 years
CDD[0:5]

In [ ]:
# annual datetimes for x axis values
time_annual=pd.date_range(tx.time.data[0],tx.time.data[-1],freq='YS')

# plot
fig=plt.figure(figsize=(15,2))
plt.plot(time_annual,CDD)
plt.title('Maximum Length of Consecutive Dry Days (CDD)')
plt.ylabel('days')
plt.show()

## Growing Season Length (GSL)

- annually, growing season starts on the first day of the first six consecutive day period where daily mean temperature is > 5C
- annually, growing season ends on the first day after 1 July of the first six consecutive day period where daily mean temperature is < 5C

Here we are inputting daily data, pulling out 2 dates per year, and calculating the number of days between the two dates.


In [ ]:
threshold = 5   # degrees C
window_len = 6  # consectutive days

# calculate mean temperature
t_mean=(tn+tx)/2

to find the start and end of the growing season we will need to "roll" through time

we'll want to group by years to find the start and end of the growing season for each year

in this case we don't need to assign a new coordinate to use .groupby, we can use .dt.year on the time coordinate to select/subset/group. 

In [ ]:
# groupby year
# let's look at what labels we get
t_mean.groupby(t_mean.time.dt.year)

let's develop our code on 1 year first, then later we'll use a loop to calculate all years

In [ ]:
# subset data to 1 year
year=1979
data_1yr=t_mean.groupby(t_mean.time.dt.year)[year]
data_1yr

In [ ]:
# make a mask for where daily temperature is greater than 5C
warm_mask=xr.where(data_1yr>threshold,1,0)
warm_mask

Now we want to partition the timeseries into rolling windows of length 6

The first window will have the first value of the data_mask (index 0) in the last position of the window plus the 5 preceeding values of the data_mask, which in this case are nan because we're at the beginning of the data_mask array

The 6th window (index 5) should be equal to the first 6 values of data_mask (no nans)

In [ ]:
# create the windows
windows=warm_mask.rolling(time=window_len,center=False).construct('window')

# print the window with the first 6 values of data_mask
print(windows.isel(time=window_len-1))

# print array info
windows

In [ ]:
# find the sum of each window
# this will tell us how many days per window are over the 5C threshold
# ignore windows that contain any nans
windows.sum('window',min_count=window_len)

Find the indexes of each 6-day window where all days were over the 5C threshold with np.where

np.where returns a tuple in this case where the resulting array is in the first index of the tuple. This is why we use the [0] to pull the array from the tuple

In [ ]:
# indexes of every window where all days are over the threshold
np.where(windows.sum('window')==window_len)[0]

But we need to adjust these indexes. Why?

Take the first value of the result above (55). This is the first window where the requirement was met (all 6 days above 5C). The windows are indexed as their last day, i.e. the 5th index window contains index days 0,1,2,3,4,5.

The start of the growing season is the first day of the first 6-day period meeting the 5C requirement, so to get the index of the first day in the window we subtract 5 (window_len-1).


In [ ]:
# find index of start of growing season
# windows are indexed to their last day, switch to first day index
gs_start_ind=np.where(windows.sum('window')==window_len)[0][0] - (window_len-1)
gs_start_ind

In [ ]:
# now let's search for the end of the growing season
# we know to only look after July 1, what index is that?
label=str(year)+'-07-01'
minval=t_mean.indexes['time'].get_loc(label)
minval

In [ ]:
# now do similar steps to find the end of the growing season

# 0/1 mask for where temperature is less than 5C
warm_mask=xr.where(data_1yr<threshold,1,0)
warm_mask

In [ ]:
# split up into 6-day windows
windows=warm_mask.rolling(time=window_len,center=False).construct('window')
windows

In [ ]:
# get the index to all the windows where temperature is always less than 5C
np.where(windows.sum('window')==window_len)[0]

In [ ]:
# windows are indexed to their last day, switch to first day index
possible_inds=np.where(windows.sum('window')==window_len)[0] - (window_len-1)
possible_inds

In [ ]:
# subset possible_inds to only indexes that correspond to days after July 1
# and take the first value in that result
gs_end_ind=possible_inds[possible_inds>minval][0]
gs_end_ind

In [ ]:
# functions to do what we did above for each year
# we'll put these functions in a loop below

def get_gs_start(data_1yr):
    mask=xr.where(data_1yr>threshold,1,0)
    windows=mask.rolling(time=window_len,center=False).construct('window')
    ind=np.where(windows.sum('window')==window_len)[0][0] - (window_len-1)
    return ind

def get_gs_end(data_1yr):
    mask=xr.where(data_1yr<threshold,1,0)
    windows=mask.rolling(time=window_len,center=False).construct('window')
    # sometimes it may be warm through the end of the year
    # in these cases we would end up with an error if no windows meet the <5C requirement
    # try/except works to pass in the last day of the year as the end of the growing season in these cases
    try:
        possible_inds=np.where(windows.sum('window')==window_len)[0]
        ind=possible_inds[possible_inds>minval][0]
    except:
        ind=364 # index of the last day of year
    return ind

In [ ]:
# group the data by year and loop through each year's worth of data
# to find the index of the start and end of the growing season 

# create empty lists for storing results
gs_start_list=[]
gs_end_list=[]

# loop through years of data
# .groupby returns an object that can be iterated over in the form (label,group_array_of_data) pairs
# here "label" are the years
# and "group" are a year's worth of data values
for label,group in t_mean.groupby(t_mean.time.dt.year):
    # call our functions and append the result to our lists
    gs_start_list.append(get_gs_start(group))
    gs_end_list.append(get_gs_end(group))

# look at the first 5 values of each list
gs_start_list[0:5], gs_end_list[0:5]

In [ ]:
# double check our work
assert all(x>=0 for x in gs_start_list), "negative values in gs_start_list"
assert all(x<=364 for x in gs_start_list), "values>364 in gs_start_list"
assert all(x>=minval for x in gs_end_list), f"values<{minval} in gs_end_list"
assert all(x<=364 for x in gs_start_list), "values>364 in gs_end_list"

In [ ]:
# calculate the growing season length for each year
# this is called a "list comprehension"
# it executes a for loop and returns results inside a list
GSL = [end_ind-start_ind for end_ind,start_ind in zip(gs_end_list,gs_start_list)] 

# # below is identical to above
# GSL=[]
# for end_ind,start_ind in zip(gs_end_list,gs_start_list):
#     GSL.append(end_ind-start_ind)

# check out the first 5 values
GSL[0:5]

In [ ]:
# using annual datetimes that we've already created for x axis values

# plot
fig=plt.figure(figsize=(15,2))
plt.plot(time_annual,GSL)
plt.title('Growing Season Length (GSL)')

fig=plt.figure(figsize=(15,2))
plt.plot(time_annual,gs_start_list)
plt.title('Start of Growing Season DOY')

fig=plt.figure(figsize=(15,2))
plt.plot(time_annual,gs_end_list)
plt.title('End of Growing Season DOY')

# Trends Analysis

Here, we will calculate the **[least squares linear regression](https://towardsdatascience.com/linear-regression-using-least-squares-a4c3456e8570)** (aka linear trend) on our climate change indicators (TNx, R99pTOT, Rx5day, CDD, and GSL) and determine if the trends are statistically significant using **[scipy.stats.linregress](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html)**.

A statistically significant trend means the slope of the linear regression line is statistically different than zero. **[scipy.stats.linregress](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html)** calculates statistical significance and provides a p value based on a two-tailed t-test. This scipy function also calculates r (correlation) from which we can get r<sup>2</sup> (coefficient of determination), a measure of the goodness of fit (or variance explained) of the linear regression. Here are two links with refreshers on linear trend **[p values](https://online.stat.psu.edu/stat501/lesson/2/2.12)** and **[r<sup>2</sup>](https://www.ncl.ac.uk/webtemplate/ask-assets/external/maths-resources/statistics/regression-and-correlation/coefficient-of-determination-r-squared.html#:~:text=6%20See%20Also-,Definition,line%20approximates%20the%20actual%20data.)**

**Big caveats:** 

1) We are calculating trends and statistical significance on only 45 years of data. In climate science, this would not be considered enough data years to robustly estimate long term change. For long term trend analysis, I personally try to use at least 80 data years, preferably 100, in order to minimize the impact of multi-decadal climate variability (**[read more about climate variability](https://climate.metoffice.cloud/climate_modes.html)**). Using even more than 100 data years would be ideal, but then you have to more carefully consider the impact of data quality and quantity. What we are calculating is the trend over a recent 45 year data period and our results would almost certainly change if we used more data years. All to say, it would be wise not to interpret our 45-year trends as the rate of change that will continue into the future. 
2) The p values provided by **[scipy.stats.linregress](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html)** or linear regression functions from other packages do not account for serial autocorrelation. For a scientific publication, you would likely need to calculate statistical significance more robustly by choosing a test statistic and calculating p values yourself that account for autocorrelation, which is beyond the scope of this workshop. And actually, most functions for linear regression (e.g. from **[xarray.DataArray.polyfit](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.polyfit.html)**, **[numpy.polynomial.polynomial.polyfit](https://numpy.org/doc/stable/reference/generated/numpy.polynomial.polynomial.polyfit.html)**, **[sklearn.linear_model.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)**) won't provide you with extra statistics like p, r, r<sup>2</sup>, so you'll have to calculate them yourself anyways if you use those functions.   


### Annual Total Precip Amount Over 99th Percentile on Wet Days (R99pTOT)

We'll start with R99pTOT because it's already in an xarray data array structure, so it will be slightly easier to work with than our climate change indicators that are stored in python lists.

In [ ]:
# remind ourselves what our data array looks like
R99pTOT

In [ ]:
# scipy.stats.linregress
# x values are the years (do not use datetimes for x, you will get incorrect results without an error thrown)
# y values are the climate change index values in xarray data array format

reg_info=ss.linregress(R99pTOT.year,R99pTOT)

# see what info is returned
reg_info

The slope of the linear regression is the 'trend'. The units of the trend will be y units/x units, which in this case is mm/year. Here we have a positive (increasing) trend of 0.609 mm/year.

Personally, I like to present my trends as units/century, which here means muliplying by 100 years/century.

In [ ]:
# save trend as a new variable and change units
trend=reg_info.slope*100 # mm/year --> mm/century
trend

In [ ]:
# if we want to plot the linear trend we will need to create the data points
# y = mx + b
regline=reg_info.slope*R99pTOT.year +reg_info.intercept
regline

In [ ]:
# plot the linear regression over the timeseries TNx

fig=plt.figure(figsize=(15,2))

R99pTOT.plot()  # the climate change indicator timeseries
regline.plot(linestyle='--')  # the linear regression line

plt.axhline(y=R99pTOT.mean(),color='grey',linestyle='dashed',linewidth=0.5) # a guide line at the data mean
plt.tick_params(labelright=True, right=True) # add ticks and tick labels to right-side axis 
plt.ylabel('mm')  # y axis label
plt.title(f'annual extreme precip (R99pTOT) with trend {trend:.2f} mm/century (r2={reg_info.rvalue**2:.3f}, p={reg_info.pvalue:.3f})') # informative title

plt.show() # show everything together

The figure above shows that the increasing trend of 60.93 mm/century in R99pTOT is meaningless because the trend is not statistically significant.

Let's check our result against a map provided by **[climdex.org](https://www.climdex.org/access/)**. What is different about the information provided on this map vs our analysis? Is R99pTOT near Starkville similar? What area of CONUS have significant R99pTOT trends?

<img src="figs/REGEN-LONG V1 2019 masked_R99pTOT_ANN_TrendMap_1972-2016_24to51_-125to-65.png"/>

### Maximum Length of Consecutive Dry Days (CDD)

In [ ]:
# remind ourselves what CDD looks like
CDD

In [ ]:
# linear regression
# x = years, y = climate change indicator values in list format
reg_info=ss.linregress(time_annual.year,CDD) 

# convert trend units
trend=reg_info.slope*100 # days/year --> days/century

# create regression line for plotting, y = mx + b
regline=reg_info.slope*time_annual.year +reg_info.intercept

In [ ]:
# plot the linear regression over the data
fig=plt.figure(figsize=(15,2))
plt.plot(time_annual.year,CDD) 
plt.plot(time_annual.year,regline,linestyle='--')
plt.axhline(y=np.asarray(CDD).mean(),color='grey',linestyle='dashed',linewidth=0.5)
plt.tick_params(labelright=True, right=True)
plt.ylabel('days')
plt.title(f'max length of consecutive dry days with trend {trend:.2f} days/century (r2={reg_info.rvalue**2:.3f}, p={reg_info.pvalue:.3f})')
plt.show()

The figure above shows that the increasing trend of 5.67 days/century in CDD is meaningless because the trend is not statistically significant.

Let's check our result against a map provided by **[climdex.org](https://www.climdex.org/access/)**. What is different about the information provided on this map vs our analysis? Is CDD near Starkville similar? What areas of CONUS have significant CDD trends?

<img src="figs/REGEN-LONG V1 2019 masked_CDD_ANN_TrendMap_1972-2016_24to51_-125to-65.png"/>

### Growing Season Length (GSL)

In [ ]:
# remind ourselves what GSL looks like
GSL

In [ ]:
# linear regression
# x = years, y = climate change indicator values in list format
reg_info=ss.linregress(time_annual.year,GSL)

# convert trend units
trend=reg_info.slope*100 # days/year --> days/century

# create regression line for plotting, y = mx + b
regline=reg_info.slope*time_annual.year +reg_info.intercept

In [ ]:
# plot the linear regression over the data
fig=plt.figure(figsize=(15,2))
plt.plot(time_annual.year,GSL) 
plt.plot(time_annual.year,regline,linestyle='--')
plt.axhline(y=np.asarray(GSL).mean(),color='grey',linestyle='dashed',linewidth=0.5)
plt.tick_params(labelright=True, right=True)
plt.ylabel('days')
plt.title(f'growing season length with trend {trend:.2f} days/century (r2={reg_info.rvalue**2:.3f}, p={reg_info.pvalue:.3f})')
plt.show()

The figure above shows that the increasing trend of 12.49 days/century in GSL is meaningless because the trend is not statistically significant.

Let's check our result against a map provided by **[climdex.org](https://www.climdex.org/access/)**. What is different about the information provided on this map vs our analysis? Is GSL near Starkville similar? What areas of CONUS have significant GSL trends?

<img src="figs/BEST (Berkeley Earth Surface Temperature) 1880-2021_GSL_ANN_TrendMap_1977-2021_24to51_-125to-65.png"/>

### TNx

In [ ]:
# remind ourselves what TNx looks like
TNx

In [ ]:
# you can't supply datetimes as the xvalues to a linear regression function
# if you use datetimes, you probably won't get an error but you'll get incorrect results
# instead of datetimes, supply integers or floats
# here we'll make a new coordinate of integer values from 0 to 539
TNx.coords['index']=('time',range(len(TNx.time)))
TNx

In [ ]:
# linear regression
# x = integers 0 to 539, y = climate change indicator values in xarray DataArray
reg_info=ss.linregress(TNx.index,TNx)

# convert trend units
trend=reg_info.slope*12*100 # C/month --> C/century

# create regression line for plotting, y = mx + b
regline=reg_info.slope*TNx.index +reg_info.intercept

In [ ]:
# plot the linear regression over the timeseries TNx
fig=plt.figure(figsize=(15,2))
TNx.plot()
regline.plot(linestyle='--')
plt.axhline(y=TNx.mean(),color='grey',linestyle='dashed',linewidth=0.5)
plt.tick_params(labelright=True, right=True)
plt.ylabel('degrees C')
plt.title(f'Monthly Warmest Night (TNx) with trend {trend:.2f} C/century (r2={reg_info.rvalue**2:.3f}, p={reg_info.pvalue:.3f})')
plt.show()

This shows that the temperature of the monthly warmest night is changing at a rate of about 3 degrees per century which is statistically significant at the 93.5% confidence level. 

Let's check our result against a map provided by **[climdex.org](https://www.climdex.org/access/)**. What is different about the information provided on this map vs our analysis? Is TNx near Starkville similar? What areas of CONUS have significant TNx trends?

<img src="figs/BEST (Berkeley Earth Surface Temperature) 1880-2019_TNx_ANN_TrendMap_1975-2019_24to51_-125to-65.png"/>

Linear regression on a timeseries with a strong seasonal signature like we see here isn't the most informative. 

It would be better to look at seasonal trends. From the TNx timeseries above we will create 4 separate timeseries, one for each season, to look at the trends in seasonal mean TNx. We'll demonstrate a single season (DJF) first and then create a loop to compute and plot all seasons. 

In [ ]:
# use resample to create a timeseries of seasonal mean values
TNx_seasonal=TNx.resample(time='QS-DEC').mean()
TNx_seasonal

In [ ]:
# we'll remove any seasonal means that were created using less than 3 values
# this only applies to the first winter season (2 values used in mean) and last winter season (1 value used in mean) since our data is Jan 1979 - Dec 2023
TNx_DJF=TNx_seasonal[0::4][1:-1] # drop first and last index

# linear regression
reg_info_DJF=ss.linregress(TNx_DJF.time.dt.year,TNx_DJF)
trend_DJF=reg_info_DJF.slope*100 # C/year --> C/century  
regline_DJF=reg_info_DJF.slope*TNx_DJF.time.dt.year +reg_info_DJF.intercept

In [ ]:
fig=plt.figure(figsize=(15,2))
TNx_DJF.plot()
regline_DJF.plot(linestyle='--')
plt.axhline(y=TNx_DJF.mean(),color='grey',linestyle='dashed',linewidth=0.5)
plt.tick_params(labelright=True, right=True)
plt.ylabel('degrees C')
plt.title(f'Monthly Warmest Night (TNx) with trend {trend_DJF:.2f} C/century (r2={reg_info_DJF.rvalue**2:.3f}, p={reg_info_DJF.pvalue:.3f})')
plt.show()


In [ ]:
# do the same thing, but for each season using a loop

season_labels=['DJF','MAM','JJA','SON']
fig=plt.figure(figsize=(15,8))

for i,season in enumerate(season_labels):
    timeseries=TNx_seasonal[i::4] # sub select to a single season

    # drop first and last index if winter
    if season=='DJF':
        timeseries=timeseries[1:-1]

    # linear regression
    reg_info_season=ss.linregress(timeseries.time.dt.year,timeseries)
    trend_season=reg_info_season.slope*100 # C/season --> C/century  
    regline_season=reg_info_season.slope*timeseries.time.dt.year +reg_info_season.intercept

    # plot
    fig.add_subplot(4,1,i+1)
    timeseries.plot()
    regline_season.plot(linestyle='--')
    plt.axhline(y=timeseries.mean(),color='grey',linestyle='dashed',linewidth=0.5)  # a horizontal reference line
    plt.tick_params(labelright=True, right=True)
    plt.ylabel('degrees C')
    plt.title(f'{season} mean warmest night with trend {trend_season:.2f} C/century (r2={reg_info_season.rvalue**2:.3f}, p={reg_info_season.pvalue:.3f})')
    
plt.tight_layout()
plt.show()

Breaking TNx down seasonally, we can see statistically significant warming trends in winter (99% confidence) and spring (~95% condfidence) warmest nights, whereas the trends in summer and fall would not be considered statistically significant.

Let's check our winter result against maps provided by **[climdex.org](https://www.climdex.org/access/)**. What is different between what we calculated and what is available at climdex.org? Can we compare our results to these maps? 

Winter 45-year trends
<table><tr>
<td> <img src="figs/BEST (Berkeley Earth Surface Temperature) 1880-2019_TNx_DEC_TrendMap_1975-2019_24to51_-125to-65.png" alt="Drawing" width="400"/> </td>
<td> <img src="figs/BEST (Berkeley Earth Surface Temperature) 1880-2019_TNx_JAN_TrendMap_1975-2019_24to51_-125to-65.png" alt="Drawing" width="400"/> </td>
<td> <img src="figs/BEST (Berkeley Earth Surface Temperature) 1880-2019_TNx_FEB_TrendMap_1975-2019_24to51_-125to-65.png" alt="Drawing" width="400"/> </td>
</tr></table>

Let's look at 100-yr TNx trends for Dec, Jan, Feb. What is different between the climdex.org trends above calculated over the 45-year period vs the 100-year period 1920-2019 below?

Winter 100-year trends
<table><tr>
<td> <img src="figs/longterm/BEST (Berkeley Earth Surface Temperature) 1880-2019_TNx_DEC_TrendMap_1920-2019_24to51_-125to-65.png" alt="Drawing" width="400"/> </td>
<td> <img src="figs/longterm/BEST (Berkeley Earth Surface Temperature) 1880-2019_TNx_JAN_TrendMap_1920-2019_24to51_-125to-65.png" alt="Drawing" width="400"/> </td>
<td> <img src="figs/longterm/BEST (Berkeley Earth Surface Temperature) 1880-2019_TNx_FEB_TrendMap_1920-2019_24to51_-125to-65.png" alt="Drawing" width="400"/> </td>
</tr></table>

### Monthly Maximum Consecutive 5-day Precipitation (Rx5day)

In [ ]:
# remind ourselves what Rx5day looks like
Rx5day

In [ ]:
# you can't supply datetimes as the x values to a linear regression function
# if you use datetimes, you probably won't get an error but you'll get incorrect results
# instead of datetimes, supply integers or floats
# here we'll make a new coordinate of integer values from 0 to 539
Rx5day.coords['index']=('time',range(len(Rx5day.time)))
Rx5day

In [ ]:
# linear regression
reg_info=ss.linregress(Rx5day.index,Rx5day)

# convert units
trend=reg_info.slope*12*100 # mm/year --> mm/century

# create trend line for plotting
regline=reg_info.slope*Rx5day.index +reg_info.intercept

In [ ]:
# plot the linear regression over the timeseries TNx
fig=plt.figure(figsize=(15,2))
Rx5day.plot()
regline.plot(linestyle='--')
plt.axhline(y=Rx5day.mean(),color='grey',linestyle='dashed',linewidth=0.5)  # a horizontal reference line
plt.tick_params(labelright=True, right=True)
plt.ylabel('mm')
plt.title(f'monthly maximum consecutive 5-day precip with trend {trend:.2f} mm/century (r2={reg_info.rvalue**2:.3f}, p={reg_info.pvalue:.3f})')
plt.show()

The figure above shows that the increasing trend of 1.11 mm/century in Rx5day is meaningless because the trend is not statistically significant.

Let's check our result against a map provided by **[climdex.org](https://www.climdex.org/access/)**. What is different about the information provided on this map vs our analysis? Is Rx5day near Starkville similar? What areas of CONUS have significant Rx5day trends?

<img src="figs/REGEN-LONG V1 2019 masked_Rx5day_ANN_TrendMap_1972-2016_24to51_-125to-65.png"/>

Let's see if there is a statistically significant trend in for any particular month

In [ ]:
# similar to how we looped through seasons for TNx, but here we use months not seasons

month_labels=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']
fig=plt.figure(figsize=(15,15))

for i,month in enumerate(month_labels):
    timeseries=Rx5day[i::12] # sub select based on month

    # linear regression
    reg_info_season=ss.linregress(timeseries.time.dt.year,timeseries)
    trend_season=reg_info_season.slope*100 # mm/year --> mm/century 
    regline_season=reg_info_season.slope*timeseries.time.dt.year +reg_info_season.intercept

    # plot
    fig.add_subplot(12,1,i+1)
    timeseries.plot()
    regline_season.plot(linestyle='--')
    plt.axhline(y=timeseries.mean(),color='grey',linestyle='dashed',linewidth=0.5)  # a horizontal reference line
    plt.tick_params(labelright=True, right=True)
    plt.ylabel('mm')
    plt.title(f'{month} maximum consecutive 5-day precip with trend {trend_season:.2f} mm/century (r2={reg_info_season.rvalue**2:.3f}, p={reg_info_season.pvalue:.3f})')
    
plt.tight_layout()
plt.show()

If we are looking for a trend that is significant at the 90% confidence level, we can see only June and November have p < 0.1, but even those months don't look robust trends. 

One quick test of robustness we can run is to hold out one or more data points at the start or end of the timeseries. If removal of a data point at one or both ends of the June and November timeseries changes p to greater than 0.1, then we can consider these trends to be not statistically significant.

In [ ]:
month_labels=['JUN','NOV']
month_ind=[5,10]
fig=plt.figure(figsize=(15,4))

for i,(m_ind,month) in enumerate(zip(month_ind,month_labels)):
    timeseries=Rx5day[i::12] # sub select based on month
    timeseries=timeseries[1:-1] # remove the first and last data point

    # linear regression
    reg_info_season=ss.linregress(timeseries.time.dt.year,timeseries)
    trend_season=reg_info_season.slope*100 # mm --> mm/century 
    regline_season=reg_info_season.slope*timeseries.time.dt.year +reg_info_season.intercept

    # plot
    fig.add_subplot(2,1,i+1)
    timeseries.plot()
    regline_season.plot(linestyle='--')
    plt.axhline(y=timeseries.mean(),color='grey',linestyle='dashed',linewidth=0.5)  # a horizontal reference line
    plt.tick_params(labelright=True, right=True)
    plt.ylabel('mm')
    plt.title(f'{month} maximum consecutive 5-day precip with trend {trend_season:.2f} mm/century (r2={reg_info_season.rvalue**2:.3f}, p={reg_info_season.pvalue:.3f})')
    
plt.tight_layout()
plt.show()

We can conclude that there are no statistically significant trends in Rx5day or in the Rx5day timeseries for any particular month.

Let's check our result against maps provided by **[climdex.org](https://www.climdex.org/access/)**. What is different about the information provided on this map vs our analysis? Is June or November Rx5day near Starkville similar? What areas of CONUS have significant June and November Rx5day trends?

<table><tr>
<td> <img src="figs/REGEN-LONG V1 2019 masked_Rx5day_JUN_TrendMap_1972-2016_24to51_-125to-65.png" alt="Drawing" width="600"/> </td>
<td> <img src="figs/REGEN-LONG V1 2019 masked_Rx5day_NOV_TrendMap_1972-2016_24to51_-125to-65.png" alt="Drawing" width="600"/> </td>
</tr></table>

# Your Turn!

**Choose one of three mini-projects below to complete on your own and prepare to share your findings**


## Option 1 (easiest)
Calculate the monthly mean daily temperature range (DTR) at Starkville, determine whether there is a statistically significant trend, and create a figure showing the DTR timeseries, linear trend, and p value. Compare your results to climdex.org, looking at the trend in DTR over CONUS for similar data years (1977-2021) and also for data years 1922-2021 from the Berkeley Earth Surface Temperature dataset. 

&emsp;Hints:
- Use daily tmax and tmin data
- Apply the relevant steps from the data cleaning section (drop leap days)
- Calculate the daily temperature range as tmax-tmin
- For each month, find the mean daily temperature range
- Determine if there is a statistically significant trend in DTR at the 90% confidence level
- Plot the DTR timeseries with its linear trend line. Your plot should have x axis tick labels as years, a y axis label that describes the units, and a title that includes the p value.
- Go to **[climdex.org](https://www.climdex.org/access/)** and get the png of DTR trend over CONUS using data years 1977-2021 from the BEST dataset
- Go to **[climdex.org](https://www.climdex.org/access/)** and get the png of DTR trend over CONUS using data years 1922-2021 from the BEST dataset

![cci_option1_figure.png](figs/cci_option1_figure.png)

## Option 2 (moderate)
Modify the TNx analysis to operate on gridded data (3-dimensional instead of the 1-dimensional data used in this notebook), performing all calculations at each grid cell. Include the relevant parts of the data cleaning section, calculate TNx, and then calculate the January TNx trend and pvalue. Present your results in a figure that shows the January TNx trend in C/year for each grid cell on a map if it is statistically significant at the 95% confidence level. Compare your results to climdex.org, looking at the trend in Jan TNx over CONUS for similar data years (1975-2019) and also for data years 1920-2019 from the Berkeley Earth Surface Temperature dataset.

&emsp;Hints:
- Use gridded tmin data for Mississippi from the file tmin_AgERA5_Mississippi_Daily_1979-2023.nc
- For data cleaning, do the steps that apply to tn only
    - nan for daily temperature greater than 70C or less than -70C
    - remove leap days
    - daily temperature outliers (5x std)
    - you may also want to use the function get_nans_per_month to plot the maximum number of nans per month found at each grid cell
- To use scipy.stats.linregress with multidimensional data, use **[xr.stack](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.stack.html)** and a for loop or use nested for loops. Alternatively, write your own function to vectorize calculation of linear trend and pvalue.
- Plot the map of Jan trend values with units C/year, but only show the trend at grid cells that are statistically significant at the 95% confidence level or greater (use the p value and **[xarray.DataArray.where](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.where.html)** to mask out insignificant trends)
- Draw state boundaries on your map with cartopy (import cartopy.crs as ccrs, import cartopy.feature as cf) or use a different method/package of your choice
- Go to **[climdex.org](https://www.climdex.org/access/)** and get the png of Jan TNx trend over CONUS using data years 1975-2019 from the BEST dataset
- Go to **[climdex.org](https://www.climdex.org/access/)** and get the png of Jan TNx trend over CONUS using data years 1920-2019 from the BEST dataset

![cci_option2_figure.png](figs/cci_option2_figure.png)

## Option 3 (moderate, longest)
Calculate the warm spell duration index (WSDI) at Starkville, determine whether there is a statistically significant trend, and create a figure showing the WSDI timeseries, linear trend, and p value. Compare your results to climdex.org, looking at the trend in WSDI over CONUS for similar data years (1977-2021) and also for data years 1922-2021 from the Berkeley Earth Surface Temperature dataset.

&emsp;Hints:
- Use daily tmax data but drop leap days entirely
- Calculate WSDI using the following criteria
    - 6 consecutive days of hot maximum temperatures
    - hot temperature threshold defined as > 90th percentile of tx for each calendar day using a centered 5-day window in the base period 1981-2010
    - warm spells that contain dates over multiple years are assigned to the year when the spell ends
- Determine if there is a statistically significant trend in DTR at the 90% confidence level
- Plot the DTR timeseries with its linear trend line. Your plot should have x axis tick labels as years, a y axis label that describes the units, and a title that includes the p value.
- Go to **[climdex.org](https://www.climdex.org/access/)** and get the png of WSDI trend over CONUS using data years 1977-2021 from the BEST dataset
- Go to **[climdex.org](https://www.climdex.org/access/)** and get the png of WSDI trend over CONUS using data years 1922-2021 from the BEST dataset

![cci_option3_figure.png](figs/cci_option3_figure.png)


<details>
    <summary><font color="blue"><b>...</b></font></summary>
  
#### Option 1 Answer Key
[climate_change_indicators_option1.ipynb](https://github.com/kerriegeil/MSU_py_training/blob/main/learn_by_doing/climate_change_indicators/assignments/climate_change_indicators_option1.ipynb)

#### Option 2 Answer Key
[climate_change_indicators_option2.ipynb](https://github.com/kerriegeil/MSU_py_training/blob/main/learn_by_doing/climate_change_indicators/assignments/climate_change_indicators_option2.ipynb)

#### Option 3 Answer Key
[climate_change_indicators_option3.ipynb](https://github.com/kerriegeil/MSU_py_training/blob/main/learn_by_doing/climate_change_indicators/assignments/climate_change_indicators_option3.ipynb)
</details>